In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitions/titanic/train.csv
/kaggle/input/competitions/titanic/test.csv
/kaggle/input/competitions/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/competitions/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/competitions/titanic/test.csv")
train_data.info()
train_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())

# 2. Define your age "buckets" (bins) and what to name them
# 0 to 2, 2 to 12, 12 to 18, 18 to 65, and 65 to 100
age_bins = [0, 2, 18, 65, 100]
category_names = ['Infant', 'Child', 'Adult', 'Senior']

train_data['Detailed_Age_Group'] = pd.cut(train_data['Age'], bins=age_bins, labels=category_names)
test_data['Detailed_Age_Group'] = pd.cut(test_data['Age'], bins=age_bins, labels=category_names)

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

features = ['SibSp', 'Parch', 'Fare', 'Sex', 'Pclass', 'Embarked', 'Detailed_Age_Group']

X = train_data[features]
y = train_data["Survived"]
X_test = test_data[features]

numerical_transformer = SimpleImputer(strategy='median')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) 
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, ['SibSp', 'Parch', 'Fare']), 
        ('cat', categorical_transformer, ['Sex', 'Pclass', 'Embarked', 'Detailed_Age_Group'])
    ])

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', XGBClassifier(random_state=42, eval_metric='logloss'))])

param_grid = {
    'model__n_estimators': [100, 300, 500],      # Number of trees
    'model__learning_rate': [0.01, 0.05, 0.1],   # How fast it learns
    'model__max_depth': [3, 5, 7]                # How deep the trees can go (XGBoost prefers shallower trees than RF!)
}

grid_search = GridSearchCV(my_pipeline, param_grid, cv=5, scoring="accuracy", n_jobs=-1)

print("Training XGBoost...")
grid_search.fit(X, y)

print(f"Best Score: {grid_search.best_score_:.4f}")
print("Best Parameters:", grid_search.best_params_)

predictions = grid_search.predict(X_test)

output = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Survived": predictions})
output.to_csv("submission.csv", index=False)
print("Saved XGBoost submission file!")

Training XGBoost...
Best Score: 0.8216
Best Parameters: {'model__learning_rate': 0.05, 'model__max_depth': 5, 'model__n_estimators': 300}
Saved XGBoost submission file!
